In [49]:
print("Function Calling")

Function Calling


In [50]:
import json
from typing import List,Dict
from openai import OpenAI
from openai.types.chat.chat_completion import ChatCompletion,ChatCompletionMessage
client : OpenAI = OpenAI()

In [55]:
def get_current_weather(location:str , unit:str = "fahrenheit")->str:
    if "lahore" in location.lower():
        return json.dumps({"location":"Lahore","temperature":"10","unit":"celsius"})
    elif "islamabad" in location.lower():
        return json.dumps({"location":"Islamabad","temperature":"5","unit":"celsius"})
    else:
        return json.dumps({"location": location, "temperature": "unknown"})
    

In [56]:
def first_request(chat:List[Dict])->ChatCompletion:
    response : ChatCompletion = client.chat.completions.create(
        messages=chat,
        model= "gpt-3.5-turbo-1106",
        tool_choice="auto",
        tools=[{
            "type":"function",
            "function":{
                "name":"get_current_weather",
                "description":" Get the current weather in a given location",
                "parameters":{
                    "type":"object",
                    "properties":{
                        "location":{
                            "type":"string",
                            "description":"The city or state like New york ,USA"
                        },
                        "unit":{
                            "type":"string",
                            "enum":["celsius","fahrenheit"]
                        }
                    },
                    "required":["location"]
                }
            }
        }] 
    )
    return response

def orchestration(message : ChatCompletionMessage ,chat:List[Dict])->bool:
    if message.tool_calls:
        
        available_function = {
            "get_current_weather" : get_current_weather
        }
        # extend conversation with assistant's reply
        chat.append(message)

        for tool_call in message.tool_calls:
            function_name = tool_call.function.name
            function_to_call = available_function[function_name]
            function_args = json.loads(tool_call.function.arguments)
            function_response = function_to_call(
                location=function_args.get("location"),
                unit=function_args.get("unit")
            )
            chat.append({
                "tool_call_id":tool_call.id,
                "role":"tool",
                "name":function_name,
                "content":function_response
            })

        return True
    
    else:
        return False

def second_request(chat:List[Dict]):
    response: ChatCompletion = client.chat.completions.create(
        messages=chat,
        model= "gpt-3.5-turbo-1106",
    )
    return response    

def run_conversation(userPrompt:str)->str:
    chat: List[Dict] =  [{
            "role":"user",
            "content":userPrompt
        }]
    ###### Step 1: First Request
    
    #send user prompt + function details to LLM
    first_response: ChatCompletion = first_request(chat)
    first_message : ChatCompletionMessage = first_response.choices[0].message
    
    ##### Step 2: Check if the model wanted to call a function then run it and update chat
    callAgain = orchestration(first_message,chat)
    
    if not callAgain:
        return first_message.content
    
    ##### Step 3: send second request to to get LLM Response in Natural language
    second_response: ChatCompletion = second_request(chat)
    return second_response.choices[0].message.content

In [57]:
run_conversation("what's the temprature in Lahore,Islamabad today")

'The temperature in Lahore today is 10°C and in Islamabad it is 5°C.'

In [58]:
run_conversation("what is 100+1?")

'100 + 1 equals 101.'

In [59]:
run_conversation("what's the temprature in Lahore,Islamabad and Kasur today")

'The current temperature in Lahore is 10°C, in Islamabad it is 5°C, and for Kasur the temperature is currently unknown.'